In [3]:
import math
import torch
import gpytorch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import random


%matplotlib inline
%load_ext autoreload
%autoreload 2

# Data import and splitting
Import input and output data and split into training and testing sets

In [4]:
inputOrig = pd.read_csv(r'CardiacSimOldDataInput.csv')
#print(inputOrig)

In [5]:
outputOrig = pd.read_csv(r'CardiacSimOldDataOutput.csv')
#print(modelOutputOrig)

In [7]:
outputOrig

,TAT_AA,TAT_VV,EDV_LV,EDP_LV,ESV_LV,pMax_LV,dpdtMax_LV,dpdtMin_LV,EDV_RV,EDP_RV,...,dpdtMax_RV,dpdtMin_RV,EDV_LA,ESV_LA,maxV_LA,maxP_LA,EDV_RA,ESV_RA,maxV_RA,maxP_RA
0,130.924,207.994,181.28,11.18,129.04,144.09,2012,-902,111.98,3.22,...,202,-81,208.18,152.21,209.16,10.22,147.79,108.32,150.33,3.82
1,146.155,210.725,117.64,3.13,81.45,100.17,1334,-615,153.20,2.23,...,618,-216,168.27,133.26,168.49,2.90,150.52,117.71,151.48,2.15
2,120.651,172.191,143.44,3.46,117.59,88.55,1399,-663,145.51,1.90,...,437,-150,162.67,131.88,166.25,5.01,145.16,114.60,147.16,3.17
3,126.374,172.024,153.41,8.31,107.61,165.29,2348,-986,105.69,2.36,...,290,-84,203.96,157.30,211.82,8.08,140.22,119.07,147.02,2.47
4,121.021,199.577,129.22,4.31,96.90,97.93,1490,-607,149.84,3.48,...,531,-173,168.89,142.26,171.16,3.84,147.05,127.12,154.24,3.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,132.674,163.557,132.22,4.78,80.88,88.33,1541,-567,119.28,3.91,...,355,-121,173.47,150.53,185.93,3.87,166.84,133.94,170.00,3.70
70,126.392,186.309,150.41,4.70,119.47,91.92,1147,-564,116.40,2.90,...,356,-149,185.58,151.93,187.47,4.59,156.02,128.30,163.50,3.36
71,131.980,170.057,147.60,8.09,94.81,105.31,1789,-647,110.37,4.08,...,335,-135,188.72,146.30,196.92,7.45,160.83,119.62,164.05,4.51
72,135.694,224.719,165.92,6.38,113.47,112.89,1666,-657,137.87,3.09,...,395,-161,174.77,136.55,186.41,6.31,147.40,109.81,153.06,4.21


In [22]:
testIndices = random.sample(range(74), 1)

In [23]:
testIndices

[7]

In [24]:
inputTest = inputOrig.iloc[testIndices,:]
inputTrain = inputOrig[~inputOrig.index.isin(testIndices)]
outputTest = outputOrig.iloc[testIndices,:]
outputTrain = outputOrig.iloc[~outputOrig.index.isin(testIndices)]

Generate Normalising Constants

In [25]:
outputTrainMean = np.mean(outputTrain,axis=0)
outputTrainStd = np.std(outputTrain,axis=0)

inputTrainMean = np.mean(inputTrain,axis=0)
inputTrainStd = np.std(inputTrain,axis=0)

Normalise Data sets

In [1]:
def standardise(data,mean,std):
    dataStandard = (data-mean)/std.T
    return dataStandard

In [2]:
inputTestNorm = standardise(inputTest,inputTrainMean,inputTrainStd)
inputTrainNorm = standardise(inputTrain,inputTrainMean,inputTrainStd)
outputTestNorm = standardise(outputTest,outputTrainMean,outputTrainStd)
outputTrainNorm = standardise(outputTrain,outputTrainMean,outputTrainStd)

NameError: name 'inputTest' is not defined

In [28]:
inputTrain

,PCa_b,trpnmax,Gncx_b,Tref,perm50,nperm,TRPN_n,dr,wfrac,TOT_A,...,a_ventricles,bt_ventricles,a_atria,bf_atria,bt_atria,k_peri,p_shift_diastasis,EDP_rv,a_lvrv,Tref_lvrv
0,0.000154,0.122142,0.001340,189.281,0.279325,2.95711,2.13423,0.145900,0.739420,18.0604,...,0.861585,4.46299,2.47469,8.58137,3.88153,0.001631,5.37553,6.18239,1.62971,0.532398
1,0.000106,0.097139,0.001098,176.706,0.450071,2.93879,2.92249,0.175040,0.288955,35.7541,...,0.753897,2.47975,1.89164,8.46425,4.20751,0.001434,8.35279,7.19977,1.38272,0.531629
2,0.000146,0.072948,0.002229,153.947,0.262872,2.94144,2.15412,0.249029,0.602822,28.7438,...,1.315720,2.36705,1.97658,5.24655,2.58736,0.000688,3.74932,7.22186,1.26298,0.811498
3,0.000092,0.074319,0.001220,139.411,0.195759,2.94652,2.79466,0.134931,0.633747,25.4347,...,1.303410,2.92956,2.47648,7.41331,2.21301,0.001480,7.21280,3.11140,1.95274,0.887341
4,0.000116,0.105088,0.001327,171.098,0.262710,3.03344,2.74316,0.224883,0.503923,32.9004,...,0.560745,3.09133,1.63551,7.84332,4.14088,0.001134,8.55608,6.29236,1.18866,0.901884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,0.000116,0.109493,0.001424,175.893,0.242235,2.41094,2.70211,0.160479,0.583591,30.2827,...,0.961820,3.78386,1.85212,11.47580,2.52115,0.000778,6.99428,6.85641,1.76490,0.757799
70,0.000125,0.101276,0.001576,186.659,0.276913,2.71953,2.78193,0.298337,0.722611,35.9240,...,1.333330,2.37173,1.81577,6.57687,1.89755,0.001445,5.23697,6.05627,1.28617,0.758834
71,0.000122,0.106121,0.001811,189.593,0.347108,2.49902,2.29310,0.165380,0.590865,34.2035,...,1.009040,1.71866,2.15164,9.32603,2.48705,0.001424,6.06730,6.98870,1.52299,0.926361
72,0.000097,0.090891,0.001469,160.713,0.235713,2.60399,2.65595,0.155777,0.527521,27.1035,...,1.214590,3.43459,2.39931,10.21610,2.13510,0.001792,3.24770,3.13479,1.09432,0.951755


In [29]:
inputTrainNorm

,PCa_b,trpnmax,Gncx_b,Tref,perm50,nperm,TRPN_n,dr,wfrac,TOT_A,...,a_ventricles,bt_ventricles,a_atria,bf_atria,bt_atria,k_peri,p_shift_diastasis,EDP_rv,a_lvrv,Tref_lvrv
0,1.365614,1.915550,-0.665475,1.221246,-0.217235,1.238948,-2.511779,-1.101794,1.464784,-1.080337,...,-0.485529,1.698868,1.443916,0.283137,1.062467,0.988225,-0.330925,0.517268,0.363570,-1.331091
1,-0.855469,0.289479,-1.332891,0.568370,2.271945,1.161657,1.172780,-0.543067,-2.032239,1.818675,...,-0.912855,-0.658088,-0.432413,0.230055,1.490162,0.487024,1.392421,1.243903,-0.566095,-1.336341
2,0.976827,-1.283731,1.781537,-0.613244,-0.457091,1.172837,-2.418807,0.875590,0.404354,0.670077,...,1.316562,-0.792025,-0.159065,-1.228292,-0.635521,-1.408319,-1.272234,1.259680,-1.016794,0.574362
3,-1.497799,-1.194601,-0.995754,-1.367932,-1.435481,1.194269,0.575265,-1.312113,0.644429,0.127900,...,1.267714,-0.123517,1.449677,-0.246259,-1.126679,0.603765,0.732553,-1.676101,1.579449,1.092152
4,-0.379622,0.806456,-0.701937,0.277211,-0.459452,1.560984,0.334539,0.412617,-0.363412,1.351112,...,-1.679318,0.068736,-1.256672,-0.051367,1.402741,-0.276435,1.510092,0.595811,-1.296532,1.191439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,-0.397639,1.092932,-0.435190,0.526160,-0.757942,-1.065336,0.142659,-0.822258,0.255061,0.922217,...,-0.087778,0.891765,-0.559594,1.594970,-0.722390,-1.179337,0.606066,0.998668,0.872423,0.207752
70,0.014752,0.558546,-0.017143,1.085115,-0.252397,0.236601,0.515761,1.821017,1.334293,1.846512,...,1.386442,-0.786463,-0.676573,-0.625356,-1.540571,0.515041,-0.411129,0.427190,-0.929507,0.214818
71,-0.101777,0.873636,0.629592,1.237444,0.770923,-0.693728,-1.769174,-0.728287,0.311531,1.564618,...,0.099599,-1.562596,0.404300,0.620637,-0.767131,0.460024,0.069496,1.093152,-0.038121,1.358547
72,-1.265020,-0.116848,-0.312089,-0.261963,-0.853022,-0.250860,-0.073107,-0.912414,-0.180218,0.401323,...,0.915260,0.476679,1.201334,1.024040,-1.228899,1.395596,-1.562590,-1.659396,-1.651626,1.531915


# Model 1: Basic GP

In [30]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ARD=True))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [31]:
class ZeroMeanGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ZeroMeanGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ARD=True))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def GPEnsemble(modelInput,modelOutput,zeroMean=False):
    models = []
    likelihoods = []
    nMod = modelOutput.shape[1]
    for i in range(nMod):
        print(i)
        likelihoods.append(gpytorch.likelihoods.GaussianLikelihood())
        if zeroMean==False:
            models.append(ExactGPModel(torch.tensor(modelInput.values), torch.tensor(modelOutput.iloc[:,i].values).squeeze(), likelihoods[i]))
        else:
            models.append(ZeroMeanGPModel(torch.tensor(modelInput.values), torch.tensor(modelOutput.iloc[:,i].values).squeeze(), likelihoods[i]))
        smoke_test = ('CI' in os.environ)
        training_iter = 2 if smoke_test else 100


        # Find optimal model hyperparameters
        models[i].train()
        likelihoods[i].train()
        X=torch.tensor(modelInput.values)
        Y=torch.tensor(modelOutput.iloc[:,i].values).squeeze()

        # Use the adam optimizer
        optimizer = torch.optim.Adam(models[i] .parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihoods[i], models[i] )


        for j in range(training_iter):
            # Zero gradients from previous iteration
            optimizer.zero_grad()
            # Output from model
            output = models[i](X)
            # Calc loss and backprop gradients
            loss = -mll(output, Y)
            loss.backward()
            #print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
             #   j + 1, training_iter, loss.item(),
              #  models[i].covar_module.base_kernel.lengthscale.item(),
               # models[i].likelihood.noise.item()
            #))
            optimizer.step()

    return models, likelihoods

NameError: name 'models' is not defined

In [59]:
def EnsembleMSE(models,modelInput,modelOutput):
    MSE = np.zeros(models.shape[0])
    nMod = modelOutput.shape[1]
    for i in range(nMod):
        models[i].eval()
        MSE.append(((models[i](torch.tensor([modelInput.values])).mean-torch.tensor(modelOutput.iloc[:,i].values))**2).mean());
    return MSE

In [60]:
def LinearEnsembleMSE(GPmodels,linearModel,modelInput,modelOutput):
    MSE = []
    nMod = modelOutput.shape[1]
    for i in range(nMod):
        models[i].eval()
        MSE.append(((models[i](torch.tensor([modelInput.values])).mean-torch.tensor(modelOutput.iloc[:,i].values))**2).mean());
    return MSE

In [61]:
inputTrain.iloc[:,0:10]

,PCa_b,trpnmax,Gncx_b,Tref,perm50,nperm,TRPN_n,dr,wfrac,TOT_A
0,0.000154,0.122142,0.001340,189.281,0.279325,2.95711,2.13423,0.145900,0.739420,18.0604
1,0.000106,0.097139,0.001098,176.706,0.450071,2.93879,2.92249,0.175040,0.288955,35.7541
2,0.000146,0.072948,0.002229,153.947,0.262872,2.94144,2.15412,0.249029,0.602822,28.7438
3,0.000092,0.074319,0.001220,139.411,0.195759,2.94652,2.79466,0.134931,0.633747,25.4347
4,0.000116,0.105088,0.001327,171.098,0.262710,3.03344,2.74316,0.224883,0.503923,32.9004
...,...,...,...,...,...,...,...,...,...,...
69,0.000116,0.109493,0.001424,175.893,0.242235,2.41094,2.70211,0.160479,0.583591,30.2827
70,0.000125,0.101276,0.001576,186.659,0.276913,2.71953,2.78193,0.298337,0.722611,35.9240
71,0.000122,0.106121,0.001811,189.593,0.347108,2.49902,2.29310,0.165380,0.590865,34.2035
72,0.000097,0.090891,0.001469,160.713,0.235713,2.60399,2.65595,0.155777,0.527521,27.1035


In [62]:
models, likelihoods = GPEnsemble(inputTrainNorm.iloc[:,0:10],outputTrainNorm)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [63]:
torch.tensor(inputTestNorm.values)

tensor([[ 1.4357e-01, -9.9045e-01,  9.5610e-01,  2.5213e-01, -6.1350e-01,
         -6.7685e-01,  7.1311e-01,  1.2650e-01, -5.0636e-01, -9.4915e-01,
          1.7364e-01, -9.4529e-01, -1.0772e+00,  1.5429e-01,  1.3533e+00,
          1.5571e+00,  6.0678e-01,  1.1445e+00,  6.4756e-01, -8.7558e-01,
          1.7058e+00,  1.2531e+00,  9.6049e-01,  6.8197e-01, -8.1675e-03,
          2.4471e-01,  6.8847e-01, -5.1804e-01,  9.0240e-01, -1.2393e+00,
         -1.1252e+00, -1.3276e-01, -1.1124e+00, -1.0201e+00, -1.5365e+00,
          8.1586e-01,  3.0395e-03,  1.9844e-01, -1.8997e-02,  8.7612e-01,
          1.6195e-01, -6.0257e-01, -1.0899e-01, -1.3849e-03, -2.2031e-01]],
       dtype=torch.float64)

In [64]:
EnsembleMSE(models,inputTestNorm.iloc[:,0:10],outputTestNorm)

[tensor(0.0071, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(2.2058, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0391, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0530, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0030, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.1093, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0109, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0872, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0177, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.3506, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.1617, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.4290, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.6811, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.2228, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0448, dtype=torch.float64, grad_fn=<MeanBackw

In [65]:
outputTestNorm.std()

TAT_AA       NaN
TAT_VV       NaN
EDV_LV       NaN
EDP_LV       NaN
ESV_LV       NaN
pMax_LV      NaN
dpdtMax_LV   NaN
dpdtMin_LV   NaN
EDV_RV       NaN
EDP_RV       NaN
ESV_RV       NaN
pMax_RV      NaN
dpdtMax_RV   NaN
dpdtMin_RV   NaN
EDV_LA       NaN
ESV_LA       NaN
maxV_LA      NaN
maxP_LA      NaN
EDV_RA       NaN
ESV_RA       NaN
maxV_RA      NaN
maxP_RA      NaN
dtype: float64

In [66]:
np.min(inputTrainNorm)

C:\Users\chris\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


PCa_b               -2.833950
trpnmax             -1.712664
Gncx_b              -1.815833
Tref                -1.772637
perm50              -1.692511
nperm               -2.127635
TRPN_n              -2.511779
dr                  -1.438047
wfrac               -2.032239
TOT_A               -1.960443
ktm_unblock         -2.575649
ca50                -1.670504
mu                  -2.345215
maxI_up             -2.175232
maxTrpn             -2.319867
g_CaL               -1.641706
Tref.1              -1.710049
perm50.1            -3.187201
nperm.1             -1.624497
TRPN_n.1            -1.328407
dr.1                -1.669282
wfrac.1             -1.597822
TOT_A.1             -1.608391
phi                 -1.736518
ca50.1              -2.661735
mu.1                -1.927379
CV_ventricles       -1.343026
k_FEC               -1.630635
CV_atria            -2.351456
k_BB                -2.325610
AV_delay            -2.012654
Rsys                -1.453499
Rpulm               -1.825437
Aol       

In [67]:
##### from SALib.sample import saltelli
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the model inputs
problem = {
    'num_vars': 45,
    'names': ['PCa_b','trpnmax','Gncx_b','Tref','perm50','nperm','TRPN_n','dr','wfrac','TOT_A','ktm_unblock','ca50','mu','maxI_up','maxTrpn','g_CaL','Tref','perm50','nperm','TRPN_n','dr','wfrac','TOT_A','phi','ca50','mu2','CV_ventricles','k_FEC','CV_atria','k_BB','AV_delay','Rsys','Rpulm','Aol','kArt','a_ventricles','bt_ventricles','a_atria','bf_atria','bt_atria','k_peri','p_shift_diastasis','EDP_rv','a_lvrv','Tref_lvrv'],
    'bounds': [[-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3],
               [-6, 3]]
}

# Generate samples
param_values = saltelli.sample(problem, 1024)
nMod = outputTrainNorm.shape[1]
Ymean = []
# Run model (example)
for j in range(nMod):
    Ymean.append(np.zeros([param_values.shape[0]]))
    Ymean[j] = models[j](torch.tensor([param_values])).mean

C:\Users\chris\AppData\Local\Temp\ipykernel_17524\3656045788.py:59: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 1024)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 45 for tensor number 1 in the list.

In [75]:
##### from SALib.sample import saltelli
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the model inputs
problem = {
    'num_vars': 10,
    'names': ['PCa_b','trpnmax','Gncx_b','Tref','perm50','nperm','TRPN_n','dr','wfrac','TOT_A'],
    'bounds': [[-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3],
               [-3, 3]]
}

# Generate samples
param_values = saltelli.sample(problem, 1024)
nMod = outputTrainNorm.shape[1]
Ymean = []
# Run model (example)
for j in range(nMod):
    Ymean.append(np.zeros([param_values.shape[0]]))
    Ymean[j] = models[j](torch.tensor([param_values])).mean

C:\Users\chris\AppData\Local\Temp\ipykernel_17524\1098596708.py:24: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 1024)


In [76]:
models[j](torch.tensor([param_values])).mean

tensor([[-0.0030, -0.0034, -0.0027,  ..., -0.0027, -0.0027, -0.0027]],
       dtype=torch.float64, grad_fn=<ViewBackward0>)

In [77]:
param_values[100]

array([-1.81933594, -2.99121094,  1.98925781,  2.78027344, -1.45605469,
       -1.60839844,  1.37988281,  1.93652344, -0.51855469, -0.17285156])

In [78]:
models[0](torch.tensor([param_values[100]])).mean

tensor([4.0964e-08], dtype=torch.float64, grad_fn=<ViewBackward0>)

In [79]:
Si = []

for i in range(nMod):
    Si.append(sobol.analyze(problem, Ymean[i].detach().numpy().squeeze(), print_to_console=True))

                   ST       ST_conf
PCa_b    1.089908e-10  2.841636e+07
trpnmax  6.200596e-04  3.913176e+13
Gncx_b   7.699717e-11  7.715692e+05
Tref     5.688771e+05  5.542766e+23
perm50   1.320151e-07  8.053754e+10
nperm    7.989391e-11  1.230993e+05
TRPN_n   5.977294e-04  1.975093e+14
dr       9.417845e-08  7.962882e+10
wfrac    7.891163e-11  5.017868e+01
TOT_A    1.095858e-08  4.893497e+09
                   S1       S1_conf
PCa_b    2.051529e-07  3.624150e+11
trpnmax  4.752646e-02  2.970134e+14
Gncx_b   4.898414e-07  7.139650e+10
Tref    -5.070221e+01  4.940091e+19
perm50  -2.514468e-05  1.485480e+13
nperm    6.338816e-07  1.644313e+10
TRPN_n  -2.302232e-03  6.789719e+14
dr      -2.006185e-05  1.744868e+13
wfrac    5.971188e-07  1.435468e+08
TOT_A    7.611523e-06  3.153909e+12
                          S2       S2_conf
(PCa_b, trpnmax)   -0.113901  3.628668e+11
(PCa_b, Gncx_b)    -0.067301  3.628668e+11
(PCa_b, Tref)      -0.067301  3.628668e+11
(PCa_b, perm50)    -0.067301  3.6286

                   ST       ST_conf
PCa_b    0.000000e+00  0.000000e+00
trpnmax  8.342355e-06  3.681620e+13
Gncx_b   0.000000e+00  0.000000e+00
Tref     4.400147e+13  2.825463e+32
perm50   0.000000e+00  0.000000e+00
nperm    0.000000e+00  0.000000e+00
TRPN_n   8.322808e-07  6.146540e+12
dr       0.000000e+00  0.000000e+00
wfrac    0.000000e+00  0.000000e+00
TOT_A    0.000000e+00  0.000000e+00
                   S1       S1_conf
PCa_b    0.000000e+00  0.000000e+00
trpnmax  8.591030e-01  8.225803e+18
Gncx_b   0.000000e+00  0.000000e+00
Tref    -3.906495e+09  2.508475e+28
perm50   0.000000e+00  0.000000e+00
nperm    0.000000e+00  0.000000e+00
TRPN_n  -7.579357e-01  5.544140e+18
dr       0.000000e+00  0.000000e+00
wfrac    0.000000e+00  0.000000e+00
TOT_A    0.000000e+00  0.000000e+00
                             S2       S2_conf
(PCa_b, trpnmax)   5.890580e+02  3.587345e+07
(PCa_b, Gncx_b)    5.890603e+02  5.571187e+02
(PCa_b, Tref)      5.890603e+02  5.595133e+12
(PCa_b, perm50)    5.890

               ST   ST_conf
PCa_b    0.582589  0.479308
trpnmax  0.680330  0.692096
Gncx_b   0.936418  0.729232
Tref     0.886617  0.727976
perm50   0.434600  0.622237
nperm    1.021542  1.484813
TRPN_n   0.966019  1.363929
dr       0.379450  0.275270
wfrac    0.368568  0.245239
TOT_A    0.374202  0.226530
               S1   S1_conf
PCa_b    0.014159  0.029686
trpnmax  0.013546  0.067263
Gncx_b   0.023108  0.037612
Tref    -0.018307  0.054983
perm50  -0.076102  0.170419
nperm    0.006552  0.033173
TRPN_n   0.028372  0.122393
dr       0.001175  0.027904
wfrac   -0.023253  0.028791
TOT_A    0.000727  0.022768
                         S2   S2_conf
(PCa_b, trpnmax)  -0.788804  1.756158
(PCa_b, Gncx_b)   -0.014772  0.158247
(PCa_b, Tref)     -0.041426  0.131434
(PCa_b, perm50)   -0.017514  0.066146
(PCa_b, nperm)    -0.004249  0.094819
(PCa_b, TRPN_n)   -0.067440  0.127733
(PCa_b, dr)       -0.029757  0.116089
(PCa_b, wfrac)    -0.083440  0.139687
(PCa_b, TOT_A)    -0.061170  0.152971
(trp

               ST   ST_conf
PCa_b    0.360637  0.117888
trpnmax  0.286932  0.080382
Gncx_b   0.393212  0.115767
Tref     0.421346  0.148543
perm50   0.250623  0.078721
nperm    0.479400  0.197284
TRPN_n   0.257571  0.075325
dr       0.310514  0.099907
wfrac    0.350677  0.101667
TOT_A    0.322382  0.088057
               S1   S1_conf
PCa_b    0.003033  0.048549
trpnmax  0.012720  0.076961
Gncx_b   0.030185  0.053598
Tref     0.019988  0.042186
perm50   0.010169  0.046984
nperm   -0.059287  0.068825
TRPN_n  -0.008942  0.043531
dr       0.003538  0.059834
wfrac    0.045066  0.040139
TOT_A    0.013150  0.072938
                         S2   S2_conf
(PCa_b, trpnmax)   0.029895  0.079239
(PCa_b, Gncx_b)    0.046117  0.061560
(PCa_b, Tref)      0.007583  0.076590
(PCa_b, perm50)    0.031143  0.054942
(PCa_b, nperm)     0.025250  0.076444
(PCa_b, TRPN_n)    0.031409  0.068701
(PCa_b, dr)        0.030509  0.058546
(PCa_b, wfrac)     0.005997  0.064652
(PCa_b, TOT_A)     0.040053  0.086836
(trp

               ST   ST_conf
PCa_b    0.040898  0.006094
trpnmax  0.056674  0.008297
Gncx_b   0.051236  0.007004
Tref     0.040848  0.005094
perm50   0.200044  0.022712
nperm    0.066892  0.009532
TRPN_n   0.215246  0.025869
dr       0.036249  0.004753
wfrac    0.377691  0.044879
TOT_A    0.173055  0.020535
               S1   S1_conf
PCa_b    0.002110  0.018224
trpnmax  0.005457  0.018095
Gncx_b   0.001407  0.019285
Tref     0.001764  0.015397
perm50   0.155045  0.037107
nperm    0.023771  0.021218
TRPN_n   0.168602  0.036954
dr       0.013636  0.014802
wfrac    0.300495  0.055272
TOT_A    0.123032  0.032557
                         S2   S2_conf
(PCa_b, trpnmax)   0.008278  0.026245
(PCa_b, Gncx_b)    0.012883  0.027635
(PCa_b, Tref)      0.010281  0.027666
(PCa_b, perm50)    0.011212  0.028142
(PCa_b, nperm)     0.016978  0.027933
(PCa_b, TRPN_n)    0.011427  0.028653
(PCa_b, dr)        0.007497  0.026532
(PCa_b, wfrac)     0.011884  0.026364
(PCa_b, TOT_A)     0.010344  0.025220
(trp

                   ST       ST_conf
PCa_b    0.000000e+00  0.000000e+00
trpnmax  4.030535e-04  6.579252e+12
Gncx_b   0.000000e+00  0.000000e+00
Tref     5.018786e+15  1.695640e+32
perm50   0.000000e+00  0.000000e+00
nperm    0.000000e+00  0.000000e+00
TRPN_n   2.488476e-05  6.486833e+11
dr       0.000000e+00  0.000000e+00
wfrac    0.000000e+00  0.000000e+00
TOT_A    0.000000e+00  0.000000e+00
                   S1       S1_conf
PCa_b    0.000000e+00  0.000000e+00
trpnmax  2.690274e+01  2.558213e+18
Gncx_b   0.000000e+00  0.000000e+00
Tref    -4.455412e+11  1.505300e+28
perm50   0.000000e+00  0.000000e+00
nperm    0.000000e+00  0.000000e+00
TRPN_n  -4.368082e+01  1.110198e+18
dr       0.000000e+00  0.000000e+00
wfrac    0.000000e+00  0.000000e+00
TOT_A    0.000000e+00  0.000000e+00
                             S2       S2_conf
(PCa_b, trpnmax)  -6.290612e+03  7.946055e+06
(PCa_b, Gncx_b)   -6.290636e+03  6.072319e+03
(PCa_b, Tref)     -6.290636e+03  3.700009e+12
(PCa_b, perm50)   -6.290

In [80]:
Si[1]

{'S1': array([ 3.49431725e-06, -2.23483901e-01,  3.09256419e-06, -2.43464729e+06,
        -1.55852156e-04,  3.15796586e-06,  4.89525912e-02, -7.98741129e-05,
         3.18599514e-06, -4.70770491e-04]),
 'S1_conf': array([1.85889562e+16, 6.05250549e+20, 4.13075197e+15, 1.55680183e+29,
        1.22125211e+19, 1.13776812e+15, 2.34346108e+21, 4.58688471e+18,
        5.44754827e+07, 1.26320788e+19]),
 'ST': array([4.73403750e-14, 1.55433112e-04, 4.72511417e-14, 2.74172760e+10,
        9.79907815e-11, 4.69543713e-14, 1.86444880e-05, 3.19130159e-11,
        4.69507373e-14, 1.03907947e-09]),
 'ST_conf': array([2.48171470e+07, 4.48772573e+16, 1.01192844e+06, 1.75316012e+33,
        8.19975668e+12, 1.47508740e+05, 6.73926440e+17, 1.76104033e+12,
        8.02783421e-01, 2.76927123e+13]),
 'S2': array([[            nan, -1.46842733e+01, -1.47091216e+01,
         -1.47091216e+01, -1.47091216e+01, -1.47091216e+01,
         -1.47091216e+01, -1.47091216e+01, -1.47091216e+01,
         -1.47091216e+01],

# Model 2: Linear regression + Zero Mean GP

In [52]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# generate random data-set

# sckit-learn implementation

# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(modeWeights, modelOutput)
# Predict
y_predicted = regression_model.predict(modeWeights)

# model evaluation
rmse = mean_squared_error(modelOutput, y_predicted)
r2 = r2_score(modelOutput, y_predicted)

# printing values
print('Slope:' ,regression_model.coef_)
print('Intercept:', regression_model.intercept_)
print('Root mean squared error: ', rmse)
print('R2 score: ', r2)


Slope: [[-1.54021589e-01  5.26176805e-01  4.17042799e-01  5.39143927e-01
  -1.09621406e-01  1.52498636e-02 -5.76630938e-02 -4.77345391e-02
  -3.48296527e-01  3.61466431e-02 -2.43841516e-02  3.96034226e-02
  -5.36768588e-02 -1.26329919e-01 -6.94980233e-02  3.14869257e-03
   1.19471093e-02 -6.49790700e-02]
 [-9.07731209e-02  8.00939349e-01  3.83192485e-01  5.35958184e-02
  -1.01739602e-01  8.98668944e-02 -2.47704147e-01  5.01152561e-02
   2.21716228e-01  1.06651020e-01  1.85600746e-05  3.69214364e-02
  -9.43340531e-02  3.05087488e-02  1.70157534e-02  3.68747010e-03
  -9.13405184e-02 -6.95339609e-04]
 [-8.31191016e-02  3.36359536e-01  4.20392750e-01  6.11489099e-01
  -1.25603466e-01 -4.53733161e-02 -5.62885097e-03 -1.15411668e-01
  -4.38773588e-01  1.27615955e-03 -4.56286778e-02  5.15934663e-02
  -6.44933969e-02 -1.00746004e-01 -6.66996083e-02  4.85151626e-02
   1.58931055e-02 -1.14531945e-01]
 [-2.03469900e-01  6.50605738e-01  3.85065482e-01  4.40049221e-01
  -8.83924406e-02  6.56633485e

In [53]:
modelOutputAdjusted = modelOutput - y_predicted

In [54]:
modelsZM, likelihoodsZM,MSEZM = GPEnsemble(modeWeights,modelOutputAdjusted,zeroMean=True)

0
1
2
3
4
5
6
7
8


In [55]:
MSEZM

[tensor(0.0053, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0077, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0145, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0096, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0038, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.2268, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0643, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0870, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0226, dtype=torch.float64, grad_fn=<MeanBackward0>)]

In [57]:
MSE

[tensor(0.0015, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.7033, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0894, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0643, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.0237, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(1.1358, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(2.0459, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(2.4015, dtype=torch.float64, grad_fn=<MeanBackward0>),
 tensor(0.5772, dtype=torch.float64, grad_fn=<MeanBackward0>)]